In [17]:
import pandas as pd
import os
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import numpy as np

In [18]:

X_train = pd.read_csv('../data/processed/X_train.csv').drop(columns = "code_snippet")
X_test = pd.read_csv('../data/processed/X_test.csv').drop(columns = "code_snippet")
y_train = pd.read_csv('../data/processed/y_train.csv').squeeze()
y_test = pd.read_csv('../data/processed/y_test.csv').squeeze()


In [19]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
boolean_features = X_train.select_dtypes(include=['bool']).columns.tolist()

In [20]:
pipeline = Pipeline(steps=[
    ('classifier', LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        max_iter=1000,
        random_state=42
    ))
])      

In [21]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)

c:\Users\Nour\Desktop\Project\Project2\code-quality-assessor\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Nour\Desktop\Project\Project2\code-quality-assessor\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
print("Classification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=["bad", "moderate", "excellent"]
))

Classification Report:
              precision    recall  f1-score   support

         bad       0.81      0.66      0.72      6899
    moderate       0.64      0.49      0.55      4240
   excellent       0.64      0.78      0.70      9923

    accuracy                           0.68     21062
   macro avg       0.69      0.64      0.66     21062
weighted avg       0.69      0.68      0.68     21062



In [23]:
# Ensure output directories exist
os.makedirs('models', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

# Save model
joblib.dump(pipeline, 'models/logistic_model.pkl')

# Save predicted labels
pd.DataFrame({'predicted_label': y_pred}).to_csv('outputs/logistic_predictions.csv', index=False)

# Save predicted probabilities
proba_df = pd.DataFrame(y_proba, columns=pipeline.named_steps['classifier'].classes_)
proba_df.to_csv('outputs/logistic_predicted_probs.csv', index=False)

print("Model, predictions, and probabilities saved.")

Model, predictions, and probabilities saved.
